In [ ]:
!pip install pafy
!pip install youtube-dl

     |████████████████████████████████| 1.9MB 6.6MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
path='/content/drive/MyDrive/soc'
output_dir= '/content/drive/MyDrive/soc'

In [ ]:
import cv2
import numpy as np
import pafy
import time
from google.colab.patches import cv2_imshow
n=2


fourcc = cv2.VideoWriter_fourcc(*'XVID')
w, h = 1280,720
fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
output_fps = 30
output_movie = cv2.VideoWriter(os.path.join(output_dir,'output_video.avi'), fourcc, output_fps, (w, h))

url = 'https://www.youtube.com/watch?v=iH1ZJVqJO3Y'
vPafy= pafy.new(url)
play = vPafy.getbest(preftype="mp4")
cap = cv2.VideoCapture(play.url)
cap.set(3,480)
cap.set(4,640)


MODEL_MEAN_VALUES = (78.4263377603, 87.7689143744, 114.895847746)

age_list = ['(0, 2)', '(4, 6)', '(8, 12)', '(15, 20)', '(25, 32)', '(38, 43)', '(48, 53)', '(60, 100)']

gender_list = ['Male', 'Female']



def load_caffe_models():

  age_net = cv2.dnn.readNetFromCaffe(os.path.join(path,'deploy_age.prototxt'), os.path.join(path,'age_net.caffemodel'))

  gender_net = cv2.dnn.readNetFromCaffe(os.path.join(path,'deploy_gender.prototxt'), os.path.join(path,'gender_net.caffemodel'))

  return (age_net, gender_net)

# Function to detetct age and gender

def video_detector(age_net, gender_net):
  # Selecting cv2 font to write on the video
  font = cv2.FONT_HERSHEY_SIMPLEX
  framecount = 0

  # Ending the loop when frames in output video are n*FPS (for e.g. 10*30=300)
  while framecount < n*output_fps:

      ret, image = cap.read()
      framecount = framecount+1

      face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_alt.xml')
      eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

      gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

      faces = face_cascade.detectMultiScale(gray, 1.1, 5)

      for (x, y, w, h )in faces:
        cv2.rectangle(image, (x, y), (x+w, y+h), (255, 255, 0), 2)

      #Get Face
        face_img = image[y:y+h, h:h+w].copy()
        blob = cv2.dnn.blobFromImage(face_img, 1, (227, 227), MODEL_MEAN_VALUES, swapRB=False)

      #Predict Gender
        gender_net.setInput(blob)
        gender_preds = gender_net.forward()
        gender = gender_list[gender_preds[0].argmax()]
        #print("Gender : " + gender)

      #Predict Age
        age_net.setInput(blob)
        age_preds = age_net.forward()
        age = age_list[age_preds[0].argmax()]
        #print("Age Range: " + age)

        overlay_text = "%s %s" % (gender, age)
        cv2.putText(image, overlay_text, (x, y), font, 1, (255, 255, 255), 2, cv2.LINE_AA)
      output_movie.write(image)
  output_movie.release()
  print(framecount)

# Running made functions
age_net, gender_net = load_caffe_models()

video_detector(age_net, gender_net)

60
